In [1]:
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import urllib.parse
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import AccessAccountsReceivable, AccessPayments
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from langchain_experimental.agents import create_pandas_dataframe_agent
from typing import Union, List, Dict
import datetime
from sqlalchemy import create_engine

In [2]:
# Langchain API KEY
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
# DATABASE Connection settings
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

# Creating conneciton to database

engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')
#conn = psycopg2.connect(f"dbname={DB_NAME} user={USERNAME} password={PASSWORD}")
accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", engine)
payments= pd.read_sql("SELECT * FROM payments", engine)

In [3]:
class Context():
    """Custom runtime context schema."""
    user_id: str

In [4]:
prompt ="""
You are a senior accountant responsible for accounts receivable reconciliation.

You have access to the following tools:
- AccessAccountsReceivable: returns the current accounts receivable as structured data.
- AccessPayments: returns all received payments as structured data.

Rules:
- When reconciliation or invoice updates are requested, you MUST call the tools.
- Always call AccessAccountsReceivable BEFORE AccessPayments.
- Never ask the user follow-up questions.
- Do not explain your reasoning.
- Do not return stringified data.
- Do not return data with different keys.

Output requirements:
- Return ONLY a Python dictionary.
- The dictionary must represent the UPDATED accounts receivable.
- Do not wrap the output in text, markdown, or code fences.
"""

In [5]:
# Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.
checkpointer = InMemorySaver()

In [6]:
tools=[AccessAccountsReceivable, AccessPayments]
model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0,
) 
model_with_tools = model.bind_tools(tools=tools) # only for OpenAI
#agent = create_agent(model=model, system_prompt=prompt, tools=tools, response_format=ToolStrategy(Union[ARResponse, PayResponse]), checkpointer=checkpointer)
#agent = create_agent(model=model, system_prompt=prompt, tools=tools, checkpointer=checkpointer)

In [7]:
# Run agent
#`thread_id` is a unique identifier for a given conversation
config = {"configurable": {"thread_id": "38"}}
# Create the agent with default parameters
agent = create_agent(model=model, system_prompt=prompt, tools=tools, checkpointer=checkpointer)

# Ask the agent a question
response = agent.invoke({"messages": [{"role": "user", 
                   "content": """
Reconcile received payments against the current accounts receivable and update invoice statuses accordingly.
Return the updated accounts receivable.
"""}]},    config=config,
    context=Context())

/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:143: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", conn)
/Users/davidal-gurnawi/Documents/data_science/agentic_test/agentic_ai_tools.py:149: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments = pd.read_sql("SELECT * FROM payments", conn)


In [ ]:
response

In [ ]:
response.keys()

In [ ]:
import re
text = (response['messages'][2].content)
text

In [ ]:
print(type(text))

In [ ]:
print(repr(text))

In [9]:
data = {'invoice_number': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14}, 
        'date': {0: datetime.date(2025, 5, 1), 1: datetime.date(2025, 9, 1), 2: datetime.date(2025, 10, 1), 3: datetime.date(2025, 1, 18), 4: datetime.date(2025, 1, 21), 5: datetime.date(2025, 1, 29), 6: datetime.date(2025, 2, 2), 7: datetime.date(2025, 2, 2), 8: datetime.date(2025, 3, 2), 9: datetime.date(2025, 5, 2), 10: datetime.date(2025, 7, 2), 11: datetime.date(2025, 10, 2), 12: datetime.date(2025, 2, 15), 13: datetime.date(2025, 2, 18)}, 
        'customer_name': {0: 'Planet Express', 1: "Mom's Friendly Robot Factory", 2: 'Romanticorp', 3: 'Hal Insitute for Criminally Insane Robots', 4: 'Cookieville Minimum-Security Orphanarium', 5: "Panucci's Pizza", 6: 'Planet Express', 7: 'Romanticorp', 8: "Malfunctioning Eddie's Rocket-Car Emporium", 9: 'Cookieville Minimum-Security Orphanarium', 10: "Mom's Friendly Robot Factory", 11: "Hal Insitute for Criminally Insane Robots", 12: "Panucci's Pizza", 13: "Unknown"}, 
        'customer_number': {0: 12038, 1: 12000, 2: 12990, 3: 12010, 4: 11900, 5: 12933, 6: 12038, 7: 12990, 8: 12230, 9: 11900, 10: 12000, 11: 12010, 12: 12933, 13: 12001}, 
        'amount': {0: 50000.0, 1: 100000.0, 2: 73640.0, 3: 12500.0, 4: 10000.0, 5: 1000.0, 6: 12300.0, 7: 50000.0, 8: 76000.0, 9: 80000.0, 10: 200000.0, 11: 12500.0, 12: 1000.0, 13: 120000.0}, 
        'due_date': {0: datetime.date(2025, 1, 18), 1: datetime.date(2025, 1, 21), 2: datetime.date(2025, 10, 1), 3: datetime.date(2025, 1, 18), 4: datetime.date(2025, 1, 21), 5: datetime.date(2025, 1, 29), 6: datetime.date(2025, 2, 2), 7: datetime.date(2025, 2, 2), 8: datetime.date(2025, 3, 2), 9: datetime.date(2025, 5, 2), 10: datetime.date(2025, 7, 2), 11: datetime.date(2025, 10, 2), 12: datetime.date(2025, 2, 15), 13: datetime.date(2025, 2, 18)}, 
        'payment': {0: 50000.0, 1: 100000.0, 2: 73640.0, 3: 12500.0, 4: 10000.0, 5: 1000.0, 6: None, 7: None, 8: None, 9: 80000.0, 10: None, 11: None, 12: None, 13: None}, 
        'payment_date': {0: datetime.date(2025, 2, 18), 1: datetime.date(2025, 1, 20), 2: datetime.date(2025, 2, 14), 3: datetime.date(2025, 1, 2), 4: datetime.date(2025, 10, 2), 5: datetime.date(2025, 2, 28), 6: None, 7: None, 8: None, 9: datetime.date(2025, 3, 13), 10: None, 11: None, 12: None, 13: None}, 
        'payment_id': {0: 302947, 1: 34847, 2: 390576, 3: 29304, 4: 3837459, 5: 39506, 6: None, 7: None, 8: None, 9: 390475, 10: None, 11: None, 12: None, 13: None}}

In [10]:
pd.DataFrame(data)

,invoice_number,date,customer_name,customer_number,amount,due_date,payment,payment_date,payment_id
0,1,2025-05-01,Planet Express,12038,50000.0,2025-01-18,50000.0,2025-02-18,302947.0
1,2,2025-09-01,Mom's Friendly Robot Factory,12000,100000.0,2025-01-21,100000.0,2025-01-20,34847.0
2,3,2025-10-01,Romanticorp,12990,73640.0,2025-10-01,73640.0,2025-02-14,390576.0
3,4,2025-01-18,Hal Insitute for Criminally Insane Robots,12010,12500.0,2025-01-18,12500.0,2025-01-02,29304.0
4,5,2025-01-21,Cookieville Minimum-Security Orphanarium,11900,10000.0,2025-01-21,10000.0,2025-10-02,3837459.0
5,6,2025-01-29,Panucci's Pizza,12933,1000.0,2025-01-29,1000.0,2025-02-28,39506.0
6,7,2025-02-02,Planet Express,12038,12300.0,2025-02-02,NaN,None,NaN
7,8,2025-02-02,Romanticorp,12990,50000.0,2025-02-02,NaN,None,NaN
8,9,2025-03-02,Malfunctioning Eddie's Rocket-Car Emporium,12230,76000.0,2025-03-02,NaN,None,NaN
9,10,2025-05-02,Cookieville Minimum-Security Orphanarium,11900,80000.0,2025-05-02,80000.0,2025-03-13,390475.0
